In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os, sys

parent = os.path.abspath(os.path.join(os.getcwd(), ".."))
if parent not in sys.path:
    sys.path.insert(0, parent)
print("Updated sys.path:", sys.path)

import method.baseline.perturb as P
import method.baseline.aggregate as A
from method.baseline.engine import OpenAIEngine

In [ ]:
system_prompt = "Answer the following math question. Think step-by-step."
example = """Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?"""

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": example},
]

In [ ]:
engine = OpenAIEngine("gpt-4o-mini")
llm_engine = "gpt-4o-mini"

In [ ]:
perturber = P.SimpleSample()
res = perturber.perturb_response(n=2, engine=engine, messages=messages)
res

In [ ]:
perturber = P.TemperaturePerturber()
res = perturber.perturb_response(n=2, engine=engine, messages=messages)
res

In [ ]:
perturber = P.SysMsgPerturber()
res = perturber.perturb_response(n=2, engine=engine, messages=messages[1:])
res

In [ ]:
perturber = P.DummyTokenPerturber()
res = perturber.perturb_response(n=2, engine=engine, messages=messages)
res

In [ ]:
perturber = P.ParaphrasingPerturber(n_inner=2)
res = perturber.perturb_response(n=4, engine=engine, messages=messages)
res

In [ ]:
aggregators = [
    A.RougeLUQ(),
    A.SbertUQ(),
    A.BertUQ(),
    A.Ask4ConfNumUQ(engine=engine),
    A.Ask4ConfWordUQ(engine=engine),
    A.LaplacianDegreeUQ(llm_engine=llm_engine),
    A.LaplacianEigenvalueUQ(llm_engine=llm_engine),
    A.SemanticVNEUq(llm_engine=llm_engine),
    A.SemanticEntropyUQ(llm_engine=llm_engine),
    A.SemanticEntropyUQ(llm_engine=llm_engine),
]
for agg in aggregators:
    print(agg.quantify_uncertainty(res))